# Flood Damage Estimates to Roads.

The purpose of this jupyter notebook is to calculate damage inflicted on roads and give a representation of the results.

The damage is measured in damage-meter. The uncertainty of the inflicted damage stems from both the uncertainty with respect to the intensity ($I$) of the flood and the uncertainty associated with the assigned damage under the given intensity ($F$). To enable the quantification of uncertainty with respect to the fragility (or assigned damage) the conditional expectation of the damage of segment $i$ ($D_i$) is estimated. That is:
$$
    E[D_i|F]
$$
The purpose is to quantify uncertainty associated with the damage mapping (as) separate from the uncertainty with respect to the flood scenarios. Fragility is sampled using a Gausian Field with an exponential kernel. The length scale (decorrelation length) has been set to 1 km.

The framework is elaborated in methods description.

## Preprocessing

There are a couple of preprocessing steps necesarry before running this notebook. Most of these are automated through a series of python scripts. Proceedure is described in `README.md`. Each script is implemented with a command line interface. Use the help function for further documentation.

NB: Recall to set parameters in `config.py`. The preprocessing scripts only apply the logging settings.

Before running this notebook, fit the damage function using the notebook `damage-function.ipynb`.

In [ ]:
import json
import os
import numpy as np

import rasterio
from rasterio.transform import rowcol
from rasterio.windows import Window
from pyproj import Proj, CRS, Transformer

import geopandas
import matplotlib.pyplot as plt
import pandas as pd

import fiona

In [ ]:
DATADIR = "/home/erlend/data/portugal-rerun"
SRCDIR = "/home/erlend/projects/gp-damage-aggregator"
os.chdir(DATADIR)
os.getcwd()

In [ ]:
# Set environment variable DATADIR
%env DATADIR $DATADIR

In [ ]:
!ls

This is the folder after preprocessing steps are finished. `src` folder is a link to the folder containing the scripts. The `random_fields` folder contains one subfolder for each set of random field files. 

In [ ]:
!ls random_fields/

In [ ]:
!ls random_fields/l-200

Note that the fields have been joined into one (virtual)-raster `random-fields.vrt` using `gdalbuildvrt`.

## Estimating damage

In [ ]:
decorrelation_length = 200
notebook_id = "l-{}".format(decorrelation_length) # for associating files with notebook and notebook-html.

# Set the geojson containing OSM-elements and the applied random fields.
args = {
    "elements_geojson" :"region-assigned.json",
    "random_fields" : "random_fields/l-{}/random_fields.vrt".format(decorrelation_length)
}

NB! If allready processed, results may be reloaded!

In [ ]:
# Read from file instead
segments_gdf = geopandas.read_file(os.path.join(DATADIR,
                                                "run/{}".format(notebook_id),
                                                "damaged_segments.shp"))

## Implementation of the damage function

The damage config is estimated and written to file in the notebook `damage-function.ipynb`

In [ ]:
with open(os.path.join(SRCDIR, 'notebooks/damage-func-config.json'), 'r') as infile:
    damage_config = json.load(infile)

In [ ]:
damage_config

The $\varepsilon$ is supposed to be sentered, hence we assume $E[\varepsilon] = 0$. This should be verified. The `d_sample` value is fitted so as to scale the size of the noise to better agree with the original step functions.

In [ ]:
class DamageSampler:
    # Adapt to current available features.
    
    def __init__(self, damage_config):
        self.beta = np.array([value for key, value in damage_config["params"].items()])
        self.eps_std = damage_config["eps"]["std"]*damage_config["d_sample"]

    def sample(self, depth, velocity, epsilon):
        # Implementation of damage function fitted in damage-function.ipynb.
        l = np.tile(self.l_hat(self.beta, depth, velocity), (epsilon.shape[0], 1)) * \
            np.exp(self.eps_std*epsilon)
        return l / (1 + l)

    def l_hat(self, beta, depth, velocity):
        return np.abs(beta[0] * depth + beta[1] * velocity + beta[2] * depth * velocity ** 2)

## Helper function to read raster values.

Below are some functions to read directly off the values from the random fields without loading everything into memory. The function is basically taken friom the script `assign_field_from_raster.py`, however it is more convenient to read directly as we estimate the damage.

In [ ]:
# Function to read values of the random field directly from raster.

def get_window(rows, cols):
        # find window
        col_off = min(cols)
        row_off = min(rows)
        width = max(cols) - min(cols) + 1
        height = max(rows) - min(rows) + 1

        window_rows = [row - row_off for row in rows]
        window_cols = [col - col_off for col in cols]

        return Window(col_off, row_off, width, height), window_rows, window_cols

def get_raster_values(dataset, feature, rastercoords_from_lonlat):
    coords = feature["geometry"]["coordinates"]
    xs, ys = rastercoords_from_lonlat.transform(*zip(*coords))
    rows, cols = rowcol(dataset.transform, xs, ys)
    # rows, cols = rowcol(dataset.transform, *zip(*coords))
    window, window_rows, window_cols = get_window(rows, cols)
    array = dataset.read(out_dtype=np.float64, window=window)
    
    # It may be problematic to evaluate outside of raster bounds.
    try:
        return array[:, window_rows, window_cols]

    except IndexError as error:
        # OSM Segment is outside of raster bounds.
        contained_in_raster = [0 <= row < dataset.shape[0] and 0 <= col < dataset.shape[1] for (row, col) in
                               zip(rows, cols)]
        rows = [row for (contained, row) in zip(contained_in_raster, rows) if contained]
        cols = [col for (contained, col) in zip(contained_in_raster, cols) if contained]
        window, window_rows, window_cols = get_window(rows, cols)
        array = dataset.read(out_dtype=np.float64, window=window)

        # append zero values outside of raster bounds.
        padded_array = np.zeros([dataset.count, len(contained_in_raster)])
        padded_array[:, contained_in_raster] = array[:, window_rows, window_cols]
        return padded_array   

# Integrate damage function

Create feature collection with damaged segments.

In [ ]:
features = ["depth", "velocity"]
return_periods = ["020", "100", "1000"] # scenarios

# feature properties of assigned elements to keep in dataframe.
keep_properties = ["id", "highway", "bridge", "lanes", "tunnel", "region"]

In [ ]:
with open(args["elements_geojson"], 'r') as file:
    flooded_elements = json.load(file)

with rasterio.open(args["random_fields"]) as dataset:
    damage_assigned = {
        "type": "FeatureCollection",
        "features": [],
    }
    
    rastercoords_from_lonlat = Transformer.from_proj(
            Proj('epsg:4326'),  # source coordinates (lonlat)
            Proj(dataset.crs),  # target coordinates
            always_xy=True  # Use easting-northing, longitude-latitude order of coordinates.
        )
    
    damage_sampler = DamageSampler(damage_config)
    
    dFi = np.flip(1/np.array(return_periods, dtype=float))  # [0.001, 0.01, 0.05]
    counter = 0 # To keep track of how many elements have been filtered.
    for element in flooded_elements['features']:
        if counter % 100 == 0:
            print("Elements processed: {}".format(counter))
        epsilon = get_raster_values(dataset, element, rastercoords_from_lonlat)
        dx = np.array(element["properties"]["deltas"])
        
        damage_meter = []
        for return_period in return_periods:
            
            # Load flood intensity parameters for each floodmap/return period 
            # Make sure that selected parameters agrees with raster band names.
            depth = np.array(element["properties"]["spatial_fields"]["depth-D312_APA_AI_T{}".format(return_period)])
            velocity = np.array(element["properties"]["spatial_fields"]["velocity-D312_APA_AI_T{}".format(return_period)])
            try:
                damage = damage_sampler.sample(depth, velocity, epsilon)
            except ValueError as error:
                print("{} - Check segment: https://www.openstreetmap.org/way/{}".format(error, element["properties"]["id"]))
            
            # Integration in space
            damage_meter.append(np.trapz(damage, dx=dx, axis=1))
        damage_arr = np.flip(np.vstack(damage_meter), axis=0)
        
        # Integration in expectation over return periods
        expected_damage_meter = np.trapz(damage_arr, dFi, axis=0)
        
        # Append filtered to damage_assigned
        out_element = {"type": "Feature", "geometry": element["geometry"], "properties": {}}
        for prop in keep_properties:
            out_element["properties"][prop] = element["properties"][prop]
        
        # Flatten list in order to load as geoDataframe
        for index, sample in enumerate(list(np.round(expected_damage_meter, 3))):
            out_element["properties"]["EDM_{}".format(index)] = sample

        out_element["properties"]["length"] = np.round(np.sum(dx), 3)
        damage_assigned["features"].append(out_element)
        counter += 1

In [ ]:
segments_gdf = geopandas.GeoDataFrame.from_features(damage_assigned, crs=4326)

# Remove bridges
segments_gdf.drop(segments_gdf[segments_gdf.bridge == "yes"].index, inplace=True)

In [ ]:
segments_gdf.columns

|column|Description
|:---|:---
|id|Open street map (OSM) id.
|highway| Road quality from OSM.
|bridge| Whether the segment is a bridge or not.
|lanes| Number of lanes.
|tunnel| Wheter the segment is a tunnel.
|region| region id number loaded from region file (below).
|length| Length of segment.
|EDM_n | Conditional expected damage meter (sampled).
|geometry | Coordinates of segment.

In [ ]:
print("Nr of flooded segments: {}".format(segments_gdf.shape[0]))

In [ ]:
segments_gdf.crs

In [ ]:
# Interactive map of flooded elements.
segments_gdf[["id", "highway", "region", "length", "geometry"]].explore()

## Damage for single element

In [ ]:
element = segments_gdf[segments_gdf.id == 132751438]

In [ ]:
element.explore()

In [ ]:
expected_dm = element.filter(regex = ("EDM_\d")).to_numpy().flatten();
print("Expected anual damage in meter: {}".format(expected_dm.mean()))

In [ ]:
plt.hist(expected_dm, bins = 50, density=True);
plt.xlabel('EDM');
#plt.title("EDM for segment {}".format(element.id.values[0]));
plt.savefig(os.path.join(SRCDIR,"notebooks/figures/edm-segment-{}-{}.png").format(element.id.values[0], notebook_id))

Recall that this is the uncertainty in expected damage due to uncertainty in the damage function. That is the lack of knowledge regarding the segments resilience to damage as computed under the given flood scenarios.

# Damage estimates for subregions.

Next, we aggregate damage according to region. Lets first plot the districts.

In [ ]:
pwd

In [ ]:
districts = "nuts/portugal_nuts.shp"
districts_gdf = geopandas.read_file(districts)

In [ ]:
districts_gdf.explore()

In [ ]:
# merge highway tags type and type_link first.
highway_type = [h_type for h_type in segments_gdf["highway"].unique() if "link" not in h_type]

for h_type in highway_type:
    segments_gdf.loc[segments_gdf.highway == "{}_link".format(h_type), "highway"] = h_type

segments_gdf["highway"].unique()

In [ ]:
# Select region by ID_1 (already written contained as column region). 
# Of course here you could run other regions as well by picking ID_1.
# Removing bridges. (Note, it might be a good idea to check validity of this tag).
santarem_gdf = segments_gdf[(segments_gdf.region == 16) & (segments_gdf.bridge != "yes")]

In [ ]:
damage_cols = [col for col in segments_gdf.columns if "EDM_" in col]

aggregate_dict = {col: "sum" for col in damage_cols}
aggregate_dict["region"] = "count"
aggregate_dict["length"] = "sum"

santarem_agg_df = santarem_gdf.groupby("highway").agg(aggregate_dict)

In [ ]:
# Number of elelements subject to flooding along with their total length (m).
santarem_agg_df[["region", "length"]]

In [ ]:
santarem_agg_df.T

In [ ]:
# filter so that we are only left with expected damage meter columns.
santarem_agg_df = santarem_agg_df.filter(regex = ("EDM_*")).T

In [ ]:
santarem_agg_df.index

In [ ]:
# Plot damage meter for primary
santarem_agg_df.primary.hist(bins=30, density=True);
#santarem_agg_df.hist(bins=30, figsize=(15,20), density=True);
plt.savefig(os.path.join(SRCDIR,"notebooks/figures/edm-region-{}-{}.png").format("santarem-primary", notebook_id))

In [ ]:
santarem_agg_df.boxplot()
plt.savefig(os.path.join(SRCDIR,"notebooks/figures/edm-santarem-box-{}.png").format(notebook_id))

In [ ]:
# Mean damage meter
santarem_agg_df.mean()

# Whole country

In [ ]:
portugal_agg_df = segments_gdf.groupby("highway").agg(aggregate_dict)

In [ ]:
portugal_agg_df.T

In [ ]:
# remove region column
portugal_agg_df = portugal_agg_df.filter(regex = ("EDM_*")).T

In [ ]:
# Histogram of expected damage meters for entire country. 
portugal_agg_df.hist(bins=30, figsize=(15,20), density=True);

#portugal_agg_df.primary.hist(bins=30, density=True);
plt.savefig(os.path.join(SRCDIR,"notebooks/figures/edm-portugal-hist-{}.png").format(notebook_id))

In [ ]:
portugal_agg_df.mean()

In [ ]:
portugal_agg_df.boxplot()
plt.savefig(os.path.join(SRCDIR, "notebooks/figures/edm-portugal-box-{}.png").format(notebook_id))

The filtering of osm elements was done to just take into consideration these type of roads (The larger ones). It is easy to add others in the analysis. In particular I see from the map that there are many roads and neighbourhoods that are not taken into the modelling. Take a look at https://wiki.openstreetmap.org/wiki/Key:highway

In [ ]:
districts_gdf

# Mean EDM by region

In [ ]:
districts_gdf.to_crs(epsg=27429, inplace=True)

In [ ]:
# Aggregate dammage by region.
damage_cols = [col for col in segments_gdf.columns if "EDM_" in col]

aggregate_dict = {col: "sum" for col in damage_cols}
aggregate_dict["region"] = "count"
aggregate_dict["length"] = "sum"

EDM_by_region_df = segments_gdf.groupby(["region", "highway"]).agg(aggregate_dict)
EDM_by_region_df.rename(columns={"region":"osm_count"}, inplace=True)

In [ ]:
# Compute the mean over all samples.
mean_EDM_by_region_df = EDM_by_region_df.filter(regex=("EDM_*")).mean(axis = 1).unstack(fill_value=0.)

In [ ]:
districts_gdf.set_index("id", inplace=True)

In [ ]:
mean_EDM_gdf = districts_gdf.join(mean_EDM_by_region_df).fillna(0.)

In [ ]:
mean_EDM_gdf[["NUTS_NAME"] + list(mean_EDM_by_region_df.columns)]

# Estimates in terms money

Road reconstruction costs will depend on a set of different factors. Som factors such as GDP per capita, oil prices may be considered to be nonlocal. Other factors, such as ground and climate conditions may be condiered as local. See the for instance [Developing Cost Estimation Models for Road Rehabilitation and Reconstruction: Case Study of Projects in Europe and Central Asia](https://www.researchgate.net/publication/273616164_Developing_Cost_Estimation_Models_for_Road_Rehabilitation_and_Reconstruction_Case_Study_of_Projects_in_Europe_and_Central_Asia)

In the following we create a very simple model based on specifying a mean, a mode and a max. However, it appears unnatural to sample these value independently for each quality of the road. I.e., we expect the prices of different qualities to be dependent. To achieve this, let $F_q$ denote the cummulative distribution of the [triangular distribution](https://en.wikipedia.org/wiki/Triangular_distribution) associated with road of quality $q$. The random cost is then given by 
$$
COST = (F_{motorway}^{-1}(U), ...,F_{trunk}^{-1}(U))
$$
where $U$ is a uniform random variable on $[0,1]$.

Note that the input numbers mean, mode and max applied refers to the average reconstruction cost predicted for the future year for portugal.

In [ ]:
# Price estimates in M€/m (million euro per meter)
os.chdir(SRCDIR)

In [ ]:
from config import COST_ROAD
pd.DataFrame.from_dict(COST_ROAD, orient="index", columns=["min", "mode", "max"])

 Price range used on cost for different road qualities in millions of euro per meter.

In [ ]:
# Specifying the inverse cummulative distribution.
from math import sqrt

def F_q_inv(a, c, b):
    F = (c-a)/(b-a)
    #return lambda z: 0.5*(a + c)*z*z + 0.5*(c-a)*z + b
    return lambda U: a + sqrt(U*(b-a)*(c-a)) if U < F  else b - sqrt((1-U)*(b-a)*(b-c))     

In [ ]:
f_dict = {key:F_q_inv(*value) for (key,value) in COST_ROAD.items()}

In [ ]:
U = np.random.uniform(0,1,10000)
cost_motorway = np.array([f_dict['motorway'](xi) for xi in U])

In [ ]:
plt.hist(cost_motorway, bins=60, density=True);

Sampling cost using a triangular distribution for motorways.

In [ ]:
# Sampling cost.
samples = 5000

U = np.random.uniform(0,1,samples)
cost_columns = ["COST_{}".format(sample) for sample in range(samples)] 
#cost_df = pd.DataFrame(np.vstack([pert(*COST_ROAD[key],samples) for key in COST_ROAD.keys()]), 
#                       index= COST_ROAD.keys(), 
#                       columns=cost_columns)
cost_df = pd.DataFrame(np.vstack([np.array([f_dict[key](xi) for xi in U]) for key in COST_ROAD.keys()]), 
                       index= COST_ROAD.keys(), 
                       columns=cost_columns)

In [ ]:
import seaborn as sns
sns.pairplot(cost_df.T)

Pairs plot of the sampled cost range for different qualities.

In [ ]:
EDM_by_region = EDM_by_region_df.filter(regex=("EDM_*")).unstack(fill_value=0.)

# Multiply cost times damage.
COST_by_region = pd.concat([EDM_by_region[d_sample].dot(cost_df) for d_sample in damage_cols],
                           axis=1,
                           keys=damage_cols,
                           names=["spatial_samples", "cost_samples"])

# Get names dictionary.
region_names = districts_gdf["NUTS_NAME"].to_dict()
COST_by_region.index.to_series().map(region_names)

Multiplying cost per meter with expected annual dammage meter yield the expected annual cost (EAC). This is done pointwise for each sample yielding a distriubution of EAC.

In [ ]:
COST_by_region.sum().to_frame()

In [ ]:
import unicodedata
COST_by_region.sum().hist(figsize=(10, 8), bins=40, density=True);
plt.savefig("notebooks/figures/eac-portugal-{}.png".format(notebook_id))

Density of total expected annual cost for the entire country in Millions of Euro.

In [ ]:
mean_COST_by_region = districts_gdf.join(pd.concat([COST_by_region.mean(axis=1), COST_by_region.std(axis=1)], axis=1, keys=["mean", "std"]),).fillna(0.)
mean_COST_by_region[["NUTS_NAME", "mean", "std"]]

In [ ]:
mean_COST_by_region[mean_COST_by_region.EAC != 0.]

In [ ]:
import contextily as cx

# Add region coordinate for tags
districts_gdf['coords'] = districts_gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
districts_gdf['coords'] = [coords[0] for coords in districts_gdf['coords']]

mean_COST_by_region = districts_gdf.join(pd.Series(COST_by_region.mean(axis=1), name="EAC")).fillna(0.)
# Skip regions with zero EAC.
mean_COST_by_region = mean_COST_by_region[mean_COST_by_region.EAC != 0.]

fig, ax = plt.subplots(1, 1, figsize=(10, 15))

mean_COST_by_region.plot(ax=ax, column="EAC", legend=True, alpha=0.8, edgecolor='k')

for idx, row in mean_COST_by_region.iterrows():
    plt.annotate(row['NUTS_NAME'], xy=row['coords'], horizontalalignment='center')

cx.add_basemap(ax,source=cx.providers.OpenStreetMap.Mapnik, crs=mean_COST_by_region.crs)

# Save run to folder

In [ ]:
os.chdir(DATADIR)
out_folder = "run/{}".format(notebook_id)
if not os.path.exists(out_folder):
    os.makedirs(out_folder )

In [ ]:
# Compute AEDM (Average Expexted Damage Meter per year)
segments_gdf["AEDM"] = segments_gdf.filter(regex = ("EDM_*")).mean(axis=1)

# Compute AEDM (Average Expexted Damage Ratio) 
segments_gdf["AEDR%"] = 100*segments_gdf["AEDM"]/segments_gdf["length"]

In [ ]:
# write geodataframe to file.
outfile = os.path.join(out_folder, "damaged_segments.shp")
#cols = ["id","highway","lanes","tunnel","region","length","AEDM","AEDR%", "geometry"]
segments_gdf.to_file(filename=outfile, driver='ESRI Shapefile')

In [ ]:
# write to files for plotting.
santarem_agg_df.to_csv(os.path.join(out_folder, "santarem_agg.csv"))
portugal_agg_df.to_csv(os.path.join(out_folder, "portugal_agg.csv"))